In [11]:
import pandas as pd

with open("cleaned.pkl", "rb") as f:
    df = pd.read_pickle(f)

In [12]:
df.head()

,text,cleaned_text,spacied,entities
0,Prisoners escaping is rare. Escapes from priso...,Prisoners escaping is rare. Escapes from priso...,"(Prisoners, escaping, is, rare, ., Escapes, fr...","[(wandsworth prison, PERSON), (the prison and ..."
1,"Part of HuffPost World News. ©2024 BuzzFeed, I...","Part of HuffPost World News. 2024 BuzzFeed, In...","(Part, of, HuffPost, World, News, ., 2024, Buz...","[(2024, DATE), (cairo, ORG), (nov 24, DATE), (..."
2,The first bombing occurred on March 2 and kill...,The first bombing occurred on March 2 and kill...,"(The, first, bombing, occurred, on, March, 2, ...","[(first, ORDINAL), (march 2, DATE), (anthony s..."
3,Trump’s latest outburst about North Korea’s nu...,Trumps latest outburst about North Koreas nuke...,"(Trumps, latest, outburst, about, North, Korea...","[(trump, PERSON), (north koreas, GPE), (armage..."
4,"Part of HuffPost News. ©2024 BuzzFeed, Inc. Al...","Part of HuffPost News. 2024 BuzzFeed, Inc. All...","(Part, of, HuffPost, News, ., 2024, BuzzFeed, ...","[(huffpost news, ORG), (2024, DATE), (aleppo b..."


In [59]:
import networkx as nx

G = nx.Graph()

In [60]:
for idx, entry in df["entities"].items():
    G.add_node(idx, bipartite=0)

    for ent_text, ent_type in set(entry):
        if ent_text not in G:
            G.add_node(ent_text, bipartite=1)
        if not G.has_edge(idx, ent_text):
            G.add_edge(idx, ent_text)

In [114]:

from collections import defaultdict

n = len(df)

Matrix = defaultdict[int, defaultdict[int, int]]
                 
matrix: Matrix  = defaultdict(lambda: defaultdict(int))

two_hop_connections = nx.all_pairs_shortest_path_length(G, cutoff=2)

for i, j in two_hop_connections:
    if isinstance(i, int):
        for num, dist in j.items():
            if dist == 2:
                matrix[i][num] += 1

In [115]:
def find_max_pair(data: Matrix):
    max_value = float('-inf')
    max_pair = (None, None)
    
    for a_key, inner_dict in data.items():
        for b_key, value in inner_dict.items():
            if value > max_value:
                max_value = value
                max_pair = (a_key, b_key)
    
    return max_pair, max_value

In [116]:
find_max_pair(matrix)

((0, 264), 1)

In [117]:
import pickle

with open("knowledge_graph.gpickle", "wb") as f:
    pickle.dump(G, f)